In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
!ls

This is an ugly workaround to allow reading of pickle files in pandas 0.19 (Misha's local version), while the files were produces in pandas 0.21 (on swan)

In [ ]:
import sys
import pandas.indexes 
sys.modules['pandas.core.indexes'] = pandas.indexes

In [ ]:
def loadInputAsDF(fin_name):
    #read original .npy files
    if '.npy' in fin_name:
        train_array = np.load(fin_name, encoding='bytes')
        train_rec_array = train_array.view(np.recarray)
        return pd.DataFrame.from_records(train_rec_array)
    elif '.pickle' in fin_name:
        return pd.read_pickle(fin_name)
    else: 
        print("I do not know how to treat this input file: {}".format(fin_name))

In [ ]:
#train_file_name = 'train10000.npy'
train_file_name = 'train_full_Nhardest5.pickle'

Read in the file properly for different file formats

In [ ]:
train_df = loadInputAsDF(train_file_name)

In [ ]:
train_df.info()

In [ ]:
train_df.corr()

## EDA

In [ ]:
#for var in ['recojet_pt', 'recojet_eta', 'recojet_phi', 'recojet_m',
#       'recojet_sd_pt', 'recojet_sd_eta', 'recojet_sd_phi', 'recojet_sd_m',
#       'n_constituents']:
#    print(var)
#    sns.jointplot(x='genjet_sd_m', y=var, data=train_df, kind='hex')

### Feature engineering and drop some columns

In [ ]:
train_df.columns

In [ ]:
columns_to_drop = []
columns_arrays = ['constituents_pt', 'constituents_eta',
       'constituents_phi', 'constituents_charge', 'constituents_dxy',
       'constituents_dz', 'constituents_Eem', 'constituents_Ehad']
columns_insignificant = ['recojet_eta', 'recojet_phi', 
       'recojet_sd_eta', 'recojet_sd_phi']
columns_insignificant_const = ['constituents_eta_0',
       'constituents_eta_1', 'constituents_eta_2', 'constituents_eta_3',
       'constituents_eta_4', 'constituents_phi_0', 'constituents_phi_1',
       'constituents_phi_2', 'constituents_phi_3', 'constituents_phi_4',
       'constituents_charge_0', 'constituents_charge_1',
       'constituents_charge_2', 'constituents_charge_3',
       'constituents_charge_4']

In [ ]:
#columns_to_drop.extend(columns_arrays)
columns_to_drop.extend(columns_insignificant)
columns_to_drop.extend(columns_insignificant_const)

In [ ]:
#To be done only if those array columns have not been droped yet
train_df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
train_df.info()

### Split and normalise

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df.drop('genjet_sd_m', axis=1), train_df['genjet_sd_m'] , test_size=0.30, random_state=314)

# XGBoost regressor

### Convert to xgboost input structure

In [ ]:
import xgboost as xgb

In [ ]:
# construct xgboost.DMatrix from numpy array, treat -999.0 as missing value
xgbmat_train = xgb.DMatrix( data=X_train, label=y_train, missing = np.nan )

In [ ]:
isinstance(xgbmat_train , xgb.DMatrix)

## Build the XGBoost model

In [ ]:
def evaluate_loss(predictions, truth):  
    #truth is xgb.DMatrix in fact, thust .get_label to get the y column
    if isinstance(truth , xgb.DMatrix):
        t = truth.get_label()
    else:
        t = truth
    ratio = predictions / t
    a = np.nanpercentile(ratio, 84, interpolation='nearest')  
    b = np.nanpercentile(ratio, 16, interpolation='nearest')  
    c = np.nanpercentile(ratio, 50, interpolation='nearest')  
    loss = (a-b)/(2.*c)  
    return loss

In [ ]:
def evaluate_loss_xgb(predictions, truth):  
    loss = evaluate_loss(predictions, truth)
    return ('xxx', loss)  

In [ ]:
#preliminary parameters. will be fine-tuned in the GridSearch
xgb_params = {'max_depth': 5, 'learning_rate':0.2, 'n_estimators':100,
              'silent':1, 'random_state': 314, 'seed': 314, 'n_job':4}

In [ ]:
clf = xgb.XGBRegressor(**xgb_params)

## Do a comparison of feature importance and extract the optimal number of trees

In [ ]:
#an adjusted function from this post: https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
def modelfit(alg, X_train, y_train, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X_train, label=y_train)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='rmse', early_stopping_rounds=early_stopping_rounds, verbose_eval=True)
        alg.set_params(n_estimators=cvresult.shape[0])
        print("Decided on {} trees".format(cvresult.shape[0]))

    
    #Fit the algorithm on the data
    alg.fit(X_train, y_train, eval_metric=evaluate_loss_xgb)
        
    #Predict training set:
    pred = alg.predict(X_train)
        
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    #Print model report:
    print("\nModel Report")
    print("RMSE : %.4g" % sqrt(mean_squared_error(y_train, pred)))
    print("Custom loss : %.4g" % evaluate_loss(y_train, pred))
    
    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
modelfit(clf, X_train, y_train, early_stopping_rounds=10)

## GridSearch to determine the optimal parameters

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.metrics import make_scorer

The next cell is CPU intense! do not try it on the full dataset!!!!!

In [ ]:
param_test1 = {'max_depth': [3,5,7],
               'min_child_weight': [1,3],
               'gamma': [0,1e-3,1e-1],
               'subsample': [0.6,0.8,1],
               'colsample_bytree':[0.6,0.8,1],
               'reg_alpha':[0, 1e-3, 1e-1],
               'reg_lambda':[1, 1e-1, 1e-3]}
gs1 = GridSearchCV(estimator=clf, param_grid=param_test1, 
                   scoring=make_scorer(evaluate_loss, greater_is_better=False),
                   n_jobs=4, cv=5)
gs1.fit(X_train, y_train)
print(gs1.best_params_)
print(gs1.best_score_)
print(gs1.grid_scores_)

In [ ]:
gs1.cv_results_

## Run XGBoost with the chosen optimal parameters

In [ ]:
#preliminary parameters. will be fine-tuned in the GridSearch
xgb_opt = {'learning_rate':0.1, 'n_estimators':50,
           'silent':1, 'random_state': 314, 'seed': 314,
           'colsample_bytree': 1, 'subsample': 1,
           'gamma': 0, 'max_depth': 7, 
           'min_child_weight': 1, 
           'reg_alpha': 0.001, 'reg_lambda': 1}

In [ ]:
clf.set_params(**xgb_opt)

In [ ]:
clf.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric=evaluate_loss_xgb,
        verbose=True)

In [ ]:
evals_result = clf.evals_result()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(clf.get_params()['n_estimators']), 
         evals_result['validation_0']['xxx'],
         'b--', label='Train')
plt.plot(range(clf.get_params()['n_estimators']), 
         evals_result['validation_1']['xxx'],
         'r-', label='Test')
plt.xlabel('N trees')
plt.ylabel('Desired metrics')
plt.legend()